## CART分类决策树

In [1]:
from collections import Counter, defaultdict
import numpy as np

In [3]:
#定义节点类，用以存放节点和节点特征
class node:
    def __init__(self, fea=-1, val=None, res=None, right=None, left=None):
        self.fea = fea  # 特征
        self.val = val  # 特征对应的值
        self.res = res  # 叶节点标记
        self.right = right # 定义左子树
        self.left = left # 定义右子树

In [18]:
#定义一个CART分类器
class CART_classify:
    #初始化
    def __init__(self, epsilon=1e-3, min_sample=1):
        self.epsilon = epsilon
        self.min_sample = min_sample  # 叶节点含有的最少样本数
        self.tree = None  #生成一棵空决策树
        
    #计算基尼系数函数定义
    def Calculate_Gini(self, y_data):
        # 计算基尼指数
        dict_t = Counter(y_data) #统计标签类型及数量，形成标签字典
        return 1 - sum([(val / y_data.shape[0]) ** 2 for val in dict_t.values()])
    
    #计算切分节点的基尼系数
    def get_Feature_Gini(self, set1, set2):
        # 计算某个特征及相应的某个特征值组成的切分节点的基尼指数
        num = set1.shape[0] + set2.shape[0] #计算总的特征数量
        return set1.shape[0] / num * self.Calculate_Gini(set2) + set2.shape[0] / num * self.Calculate_Gini(set2) #返回切分节点的基尼系数
    
    
    
    def bestSplit(self, splits_set, X_data, y_data):
        # 返回所有切分点的基尼指数，以字典形式存储。键为split，是一个元组，第一个元素为最优切分特征，第二个为该特征对应的最优切分值
        pre_gini = self.get_Feature_Gini(X_data, y_data)
        subdata_inds = defaultdict(list)  # 切分点以及相应的样本点的索引
        for split in splits_set:
            for index, sample in enumerate(X_data): #enumerate函数生成一个枚举类型，包含X_data的元素及其下标，可以被遍历
                if sample[split[0]] == split[1]:
                    subdata_inds[split].append(index) #形成字典
        min_gini = 1
        best_split = None
        best_set = None
        for split, data_index in subdata_inds.items(): #循环遍历subdata_inds的键值对
            set1 = y_data[data_index]  # 满足切分点的条件，则为左子树
            set2_inds = list(set(range(y_data.shape[0])) - set(data_index))
            set2 = y_data[set2_inds]
            if set1.shape[0] < 1 or set2.shape[0] < 1:
                continue
            now_gini = self.get_Feature_Gini(set1, set2)
            if now_gini < min_gini: #小于最小基尼系数，则更新最小基尼系数
                min_gini = now_gini
                best_split = split #同时更新最佳切分 
                best_set = (data_index, set2_inds) #同时更新最佳叶节点
        if abs(pre_gini - min_gini) < self.epsilon:  # 若切分后基尼指数下降未超过阈值则停止切分
            best_split = None
        return best_split, best_set, min_gini    
    

    def buildTree(self, splits_set, X_data, y_data):
        if y_data.shape[0] < self.min_sample:  # 数据集小于阈值(只有)直接设为叶节点
            return node(res=Counter(y_data).most_common(1)[0][0])
        best_split, best_set, min_gini = self.bestSplit(splits_set, X_data, y_data)
        if best_split is None:  # 基尼指数下降小于阈值，则终止切分，设为叶节点
            return node(res=Counter(y_data).most_common(1)[0][0]) #返回标签数最多的一类作为该叶节点的标识
        else:  #若基尼系数未小于阈值，则递归改函数，直至小于阈值
            splits_set.remove(best_split)
            left = self.buildTree(splits_set, X_data[best_set[0]], y_data[best_set[0]])
            right = self.buildTree(splits_set, X_data[best_set[1]], y_data[best_set[1]])
            return node(fea=best_split[0], val=best_split[1], right=right, left=left) #修改左右子树
        
    #训练函数
    def train(self, X_data, y_data):
        # 训练模型，CART分类树与ID3最大的不同是，CART建立的是二叉树，每个节点是特征及其对应的某个值组成的元组
        # 特征可以多次使用
        splits_set = [] #待切分为空
        for fea in range(X_data.shape[1]):   #索引特征矩阵的每一列
            unique_vals = np.unique(X_data[:, fea]) #将每一列（特征列）解析成列表
            if unique_vals.shape[0] < 2: #如果该类特征值个数小于两个，则无需切分
                continue
            elif unique_vals.shape[0] == 2:  # 若该类特征值个数只有2个，则只有一个切分点，非此即彼
                splits_set.append((fea, unique_vals[0])) #将特征标签和特征值以元组的形式添加进待切分列表
            else:    # 若特征取值大于2个
                for val in unique_vals:
                    splits_set.append((fea, val)) #将特征标签和特征值以元组的形式添加进待切分列表
        self.tree = self.buildTree(splits_set, X_data, y_data) #构建决策树
        return
    
    def predict(self, x):
        def helper(x, tree):
            if tree.res is not None:  # 表明到达叶节点
                return tree.res
            else:
                if x[tree.fea] == tree.val:  # "是" 返回左子树
                    branch = tree.left
                else:
                    branch = tree.right
                return helper(x, branch)

        return helper(x, self.tree)


In [22]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split


if __name__ == '__main__':
    #加载sklearn中的鸢尾花数据集
    X_data = load_iris().data 
    y_data = load_iris().target

    #划分训练集和测试集
    X_data_train, X_data_test, y_data_train, y_data_test = train_test_split(X_data, y_data, test_size=0.3, random_state=1)
    classifier = CART_classify() #实例化一个分类器
    classifier.train(X_data_train, y_data_train) #训练分类器
    score = 0
    for X, y in zip(X_data_test,y_data_test):
        if classifier.predict(X) == y: #计算精度
            score += 1 #预测正确则加1
    print('accuracy is {}'.format(score / len(y_data_test)))


accuracy is 0.8888888888888888


In [45]:
## 小测试
test = [1.5, 2.9, 4.6, 6.6]
print('预测类型为第{}类'.format(classifier.predict(test)))

预测类型为第2类
